In [24]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
from datetime import date
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoData, LayersControl

import ipywidgets as widgets 
from ipywidgets import AppLayout, Button, Layout, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, DatePicker, Output, VBox, HBox, Text, SelectMultiple, Combobox, Accordion
from ipywidgets.embed import embed_data
from IPython.display import clear_output, HTML, display, Image

import threading
import time
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import datetime
import itertools

from app import *

In [25]:
# HEADER
header = widgets.HTML("<h1>OSM Data Change Tracker</h1>", layout=Layout(height='auto'))
header.style.text_align='center'

footer = widgets.HTML('<h5 style="color:DodgerBlue;"> Source: OpenStreet Map via Overpass API </h5>', layout=Layout(height='auto'))

# DATE PICKER
start_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)
end_dt = DatePicker(layout=Layout(flex='1 1 0%', width='auto'), disabled=False)

# Map coord 
map_coord = widgets.Text(
    value= '(37.2367241, 131.8738459)',
    description='Map Center:',
    disabled=False, style=dict(description_width='initial'),
    continuous_update=False
    )

# Map Zoom
map_zoom = widgets.IntText(
    value=15,
    description='Zoom:',
    disabled=False,
    continuous_update=False
)

# BBOX 
bbox_coord = widgets.Text(
    value= '(37.22828,131.85043,37.25282, 131.88316)',
    description='Bbox Coord:',
    disabled=False, style=dict(description_width='initial')
    )

# Filenam
input_filename = widgets.Text(
    value='sample.csv',
    placeholder='filename',
    description='Save File as:',
    disabled=False, style=dict(description_width='initial')
    )

# SPECIFY INTERVAL
hourly_slider = widgets.IntSlider(
    value=24,
    min=1,
    max=200,
    description='Hourly Interval:'
)

# REQUEST BUTTON 
Request_button = widgets.Button(
    description='Download Data',
    disabled=False,
    button_style='success',
    icon='table', layout = Layout(width='200px', height='auto'),
    style=dict(description_width='initial'))


# BBOX Refresh button
bbox_button = widgets.Button(
    description='Update',
    disabled=False,
    button_style='warning',
    icon='arrow-rotate-right', layout = Layout(width='80px', height='auto'),
    style=dict(description_width='initial'))

progress_search = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, 
    description='Progress',
    bar_style='info',
    style={'bar_color': '#94b79f'})
Complete_msg = widgets.HTML()


In [29]:
def tab1_on_value_change(change):
    clear_output()
    feature_collection = getEmptyFeatureCollection()
    tab1 = HBox(children=[VBox(children=[tab1_header, HBox(children=[Label(value='Start Date', layout=dict(height='auto')), 
                                                                 start_dt, Label(value='End Date', layout=dict(height='auto')), end_dt]), 
                                     hourly_slider, 
                                     HBox(children=[bbox_coord, bbox_button]), 
                                     input_filename, 
                                     Request_button, 
                                     HBox(children=[progress_search, Complete_msg]), footer]), 
                                     VBox(children=[HBox(children=[map_coord, map_zoom]), createMap(eval(map_coord.value), map_zoom.value, draw_control)])])
    display(tab1) 


def tab2_on_value_change(change):
  df = pd.read_csv(os.path.join('../data', selection_filename.value))
  with plot_output: 
    clear_output()
    getHistPlot(os.path.join('../data', selection_filename.value), 'chart.png')
    img = Image(os.path.join('../figs', 'chart.png'))
    display(img)
  with table_output:
    clear_output() 
    counter_df = osmIDCounter(os.path.join('../data', selection_filename.value)).head(10).reset_index(drop = True)
    display(HTML(counter_df.to_html(render_links=True, escape=False)))

In [30]:
feature_collection = getEmptyFeatureCollection()
draw_control = getDrawControl()

Request_button.on_click(callback)
bbox_button.on_click(updateBBOX)
map_coord.observe(tab1_on_value_change, names='value')
map_zoom.observe(tab1_on_value_change, names='value')


In [31]:
plot_output = widgets.Output(layout = Layout(width='600px', height='auto'))
table_output = widgets.Output(layout = Layout(width='400px', height='auto'))

selection_filename = widgets.Dropdown(
    options=get_files(os.path.join('../data')),
    description='Select File:',
    style=dict(description_width='initial')
)

%matplotlib inline


selection_filename.observe(tab2_on_value_change, names='value')

In [32]:
tab1_header = widgets.HTML(value='<h2>OSM Change Downloader</h2>')
tab1 = HBox(children=[VBox(children=[tab1_header, HBox(children=[Label(value='Start Date', layout=dict(height='auto')), 
                                                                 start_dt, Label(value='End Date', layout=dict(height='auto')), end_dt]), 
                                     hourly_slider, 
                                     HBox(children=[bbox_coord, bbox_button]), 
                                     input_filename, 
                                     Request_button, 
                                     HBox(children=[progress_search, Complete_msg]), footer]), 
                                     VBox(children=[HBox(children=[map_coord, map_zoom]), createMap(eval(map_coord.value), map_zoom.value, draw_control)])])


In [33]:
tab1

In [34]:
tab2_header = widgets.HTML(value='<h2>Data Visualizer</h2>')
table_header = widgets.HTML(value='<h2>Top 10 OSM IDs</h2>')

tab2 = HBox(children=[VBox(children=[tab2_header, selection_filename, table_output]), plot_output])

In [35]:
tab2